<p style="font-size: 30px; font-weight: bold;">Memory Model</p>

C++ concurrency mechanisms rely on language guarantees known as **memory model**

Answers questions such as
* Can two variables be updated concurrently without interference?
* In which order does a thread observe the writes of other threads?

# Order of updates

Can be influenced by two factors
* Order in which memory request “messages” arrive not necessarily the same as the one in which they were issued
* A single thread may observe writes in an order different from the order another thread wrote them
* Order may even differ among multiple readers
* Compiler may re-order instructions (even possible on uni-processor systems)

# Objects and Memory locations

All data in a C++ program is made up of objects
* An object is a region of storage
* Every variable is an object, including those that are members of other objects
* Every object occupies at least one memory location
* Variables of fundamental types such as int or char are exactly one memory location, whatever their size, even if they are adjacent or part of an array
* Adjacent bit fields are part of the same memory location
* The C++ memory model guarantees that we can update two separate memory locations without interference

<img src="img/obj_mem_alloc.png" alt="Object memory allocation example" width="80%" style="margin: 0 auto;">

# Atomic Operations

**Data race** – if there is no enforced ordering between two accesses to a single memory location from separate threads, one or both of the accesses is non-atomic, and one or both is a write

* An atomic operation is an indivisible operation
    * Cannot be observed half-done
    * Either done or not done
* Non-atomic operations may be observed half-done by another thread
    * Can cause data race
* Non-atomic accesses can be ordered around an atomic operation
    * Foundation for data protection and synchronization mechanisms

<img src="img/half_written_array.png" alt="Object memory allocation example" width="80%" style="margin: 0 auto;">

## Standard atomic types

* Assignment from and implicit conversion to the corresponding built-in types
* Direct `load()` and `store()` member functions, `exchange()`, `compare_exchange_weak()`, and `compare_exchange_strong()`
* Compound assignment operators where appropriate: `+=`, `-=`, `*=`, `|=`, etc. and corresponding named member functions such as `fetch_add()`

## `compare_exchange_strong()`

<img src="img/compare_exchange_strong.png" alt="Compare Exchange Strong flow" width="80%" style="margin: 0 auto;">

## `compare_exchange_weak()`

<img src="img/compare_exchange_weak.png" alt="Compare Exchange weak flow" width="80%" style="margin: 0 auto;">

In [17]:
// FIRST CASE => b = false before

#include <atomic>
#include <assert.h> 
#include <iostream>

std::atomic<bool> b = false;
bool expected = false;
// Assure this value is the desired
while (!b.compare_exchange_weak(expected, true) && !expected) {
    std::cout << "In Loop wtih expected: " << expected << std::endl;
}
if (b == true) std::cout << "The system return the expected value\n";

The system return the expected value


In [18]:
// SECOND CASE => b = true before

#include <atomic>
#include <assert.h> 
#include <iostream>

std::atomic<bool> b = true;
bool expected = false;
// Assure this value is the desired
while (!b.compare_exchange_weak(expected, true) && !expected) {
    std::cout << "In Loop wtih expected: " << expected << std::endl;
}
if (b == true) std::cout << "The system return the expected value\n";

The system return the expected value


## Atomic pointer arithmetic

* Atomic form of a pointer is `std::atomic<T*>`
* Interface similar to `std::atomic<bool>` 
* Additional operations for pointer arithmetic
* `+=`, `-=`, pre- and post-increment and decrement `++`, `--`
* `fetch_add()`, `fetch_sub()` – return original value (read-modify-write operation)

<img src="img/atomic_pointer_op.png" alt="Atomic Pointer operations example" width="80%" style="margin: 0 auto;">

# Defferent views on the Multiverse

<img src="img/multiverse_example.png" alt="Multiverse example" width="80%" style="margin: 0 auto;">

* The different universes each constitute a separate memory location that is shared between all threads
* You and your friends constitute threads that access these memory locations
* Traveling to a universe is the same as reading or writing a memory location
* The history of a universe is the same as the history of a memory location

# Memory Order

## Sequential Consistency

* Simplest memory order
* Every thread sees the effects of every operation in the same order. There are only two possibilities that an atomic operation A *happens-before* B or an operation B *happens-before* A. This operations can be in different threads and there are no problem in that case.
    * As if the operations of all the threads were executed in some sequential order (like by a single thread)
    * The operations of each individual thread appear in this sequence in the order specified by the program
* Many possible sequentially consistent orders for a given set of threads

### Principles to satisfy sequential consistency

1. Method calls should appear to happen in a one-at-a-time, sequential order
2. Method calls should appear to take effect in program order. 
    * Program order is the order in which a single thread issues method calls. Method calls by different threads are unrelated by program order.

It does not matter that there are more possible representations, if we found **one** that fulfills what we are seeing in the history, it is **sequential consistency**

<img src="img/multiverse_example_2.jpg" alt="Multiverse example with FIFO" width="80%" style="margin: 0 auto;">

## Synchronizing operations and enforcing ordering

<img src="img/ordering_with_atomics.png" alt="Compare Exchange weak flow" width="80%" style="margin: 0 auto;">

## Sequenced-before

* Within the same thread, evaluation A may be sequenced-before evaluation B according to the C++ evaluation order
* Between different threads, an evaluation A of one thread may `synchronize-with` an evaluation of another thread, or they might be `dependency-ordered-before`

## Syncrohinzes-with relationship

The synchronizes-with relationship is something that you can get only between operations on atomic types. Operation `A` synchronizes-with operation `B`, if `A` is a store to some atomic variable `m`, with an ordering of `std::memory_order_release`, or `std::memory_order_seq_cst`, `B` is a load from the same variable `m`, with an ordering of `std::memory_order_acquire` or `std::memory_order_seq_cst`, and `B` reads the value stored by `A`.

## Dependency-before relationship

Dependency-ordered-before does not affect other variables other than the one with the atomic memory order imposed. This is the big difference with `synchronizes-with` relationship, where this is a rule of this relationship. In the case of the example, the variable `a` could be fire because there no relationship with the `while` loop

<img src="img/dependency-ordered-before-2.png" alt="dependency-ordered-before.png" width="80%" style="margin: 0 auto;">

<img src="img/dependency-ordered-before.png" alt="dependency-ordered-before.png" width="80%" style="margin: 0 auto;">

## Relationships between evaluations

<img src="img/sequence_before.png" alt="Sequence before" width="80%" style="margin: 0 auto;">

<img src="img/synchronizes-with.png" alt="relationships with synchronizes-with" width="80%" style="margin: 0 auto;">

## Inter-thread happens-before and Happens-before relationship

### Inter-thread happens-before

Between threads, evaluation A inter-thread happens before evaluation B if any of the following is true

* A synchronizes-with B
* A is dependency-ordered before B
* A synchronizes-with some evaluation X, and X is sequenced-before B
* A is sequenced-before some evaluation X, and X inter-thread happens-before B
* A inter-thread happens-before some evaluation X, and X inter-thread happens-before B

### Happens-before

Regardless of threads, evaluation A happens-before evaluation B if any of the following is true:

* A is sequenced-before B
* A inter-thread happens before B

If one evaluation modifies a memory location, and the other reads or modifies the same memory location, and if at least one of the evaluations is not an atomic operation, the behavior of the program is undefined (the program has a data race) unless there exists a happens-before relationship between these two evaluations.

## Memory-ordering semantics

Organize as lower to high cost

* `relaxed`: Relax operation
* `acquire`: A load operation with this memory order performs the acquire operation on the affected memory location
* `release`: A store operation with this memory order performs the release operation
* `acq_rel`: A read-modify-write operation with this memory order is both an acquire operation and a release operation
* `seq_cst`: Sequentially consistent operation aka like a Single thread

<img src="img/table_memory_ordering.png" alt="Table Memory Ordering" width="80%" style="margin: 0 auto;">

## Release-acquire ordering

* Still no total ordering but some synchronization
* Atomic stores are release operations (memory_order_release)
* Atomic reads are acquire operations (memory_order_acquire)
* Atomic read-modify-write operations are either acquire, release, or both
* Synchronization is pairwise – between the thread that does the release and the thread that does the acquire
* Use case – lock synchronization

<img src="img/rel_acq_statement.png" alt="Release-Acquire Statement" width="60%" style="margin: 0 auto;">

* Release and acquire operations must be paired up
* The value stored by a release operation must be seen by an acquire operation for either to have any effect

Once the atomic load is completed, thread B is guaranteed to see everything thread A wrote to memory

## Release Sequence

After a release operation A is performed on an atomic object M, the longest continuous subsequence of the modification order of M that consists of: 
* Writes performed by the same thread that performed A [until C++20]
* Atomic read-modify-write operations made to M by any thread

is known as `release sequence` headed by A

<img src="img/release_sequence.png" alt="Release Sequence example" width="80%" style="margin: 0 auto;">

An atomic operation A that performs a release operation on an atomic object M synchronizes-with an atomic operation B that performs an
acquire operation on M and takes its value from any side effect in the release sequence headed by A.

<img src="img/ordering_non_atomics.png" alt="Release Sequence example" width="80%" style="margin: 0 auto;">